In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import sin, cos, sqrt, exp
import yfinance as yf
from datetime import datetime, timedelta
import json

def trigfit(x, n, w, m, a, b, xm):
    Sc =0.0
    Ss =0.0
    Scc=0.0
    Sss=0.0
    Scs=0.0
    Sx =0.0
    Sxc=0.0
    Sxs=0.0
    for i in range(n):
        c = cos(w*i)
        s = sin(w*i)
        dx = x[i] - xm[i]
        Sc +=c
        Ss +=s
        Scc+=c*c
        Sss+=s*s
        Scs+=c*s
        Sx +=dx
        Sxc+=dx*c
        Sxs+=dx*s
    Sc /=n
    Ss /=n
    Scc/=n
    Sss/=n
    Scs/=n
    Sx /=n
    Sxc/=n
    Sxs/=n   
    if w == 0 or None:
        m = Sx
        a = 0.0
        b = 0.0
    else:
        den=(Scs-Sc*Ss)**2-(Scc-Sc*Sc)*(Sss-Ss*Ss)
        a=((Sxs-Sx*Ss)*(Scs-Sc*Ss)-(Sxc-Sx*Sc)*(Sss-Ss*Ss))/den
        b=((Sxc-Sx*Sc)*(Scs-Sc*Ss)-(Sxs-Sx*Ss)*(Scc-Sc*Sc))/den
        m=Sx-a*Sc-b*Ss
    return w, m, a ,b 
def freq(x, n, w, m, a, b, xm):
    FreqTOL =0.00001
    z = [None] * n
    alpha = 0.0 # = beta for initialization
    beta = 2.0
    z[0] = x[0] - xm[0]
    count = 0
    while abs(alpha - beta) > FreqTOL:
        alpha = beta
        z[1] = x[1] = xm[1] + alpha * z[0]
        num = z[0] * z[1]
        den = z[0] * z[0]
        for i in range(2, n):
            z[i] = x[i] - xm[i] + alpha*z[i-1] - z[i-2]
            num += z[i-1]*(z[i]+z[i-2])
            den+=z[i-1]*z[i-1]
        beta = num/den
        count += 1
        if count >= 1000:
            break
    if -1 <= beta/2.0 <= 1:
        w = np.arccos(beta/2.0)
    else:
        w= 0 
    w, m, a, b = trigfit(x, n, w, m, a, b, xm)
    return w, m, a, b


def main(adj_close):
    '''
    adj_close should be an array
    '''
    shop = adj_close
    av = 0.0

    past = 252
    pred = 100
    # past closed price
    x = [None] * past
    close = shop
    close = close[::-1]
    # average price for the past 300(past) days
    xm = [None] * past
    ym = [None] * 101
    for i in range(past):
        x[i] = close.iloc[i]
        av += x[i]
    av/=past

    for i in range(past):
        xm[i] = av
        if(i <= pred):
            ym[i] = av
    harmonics = 20
    w = 0.0
    m = 0.0
    a = 0.0
    b = 0.0
    for i in range(1, harmonics + 1):
        w, m, a, b = freq(x, past, w, m, a, b, xm)
        for i in range(past):
            xm[i] += m+a*cos(w*i) + b*sin(w*i)
            if(i <= pred):
                ym[i] += m+a*cos(w*i)-b*sin(w*i)

    return ym

In [38]:
import pandas as pd
from tqdm import tqdm
adj_close = pd.read_parquet("../database/Adj_close.parquet")
adj_close = adj_close[adj_close.index >= '2020-01-01']

In [42]:
factor_value = pd.DataFrame()
iter_ticker_list = list(adj_close.columns)
for i in tqdm(range(len(adj_close)-252), desc='looping by date, FFT: '):
    close_temp = adj_close[i:i+252]
    list_factor = [0] * len(iter_ticker_list)
    date = close_temp.index[-1]
    for index, ticker in enumerate(iter_ticker_list):


        ym = main(close_temp[ticker])
        print(ticker, date, ym)
        r = np.polyfit(np.arange(0, len(ym), 1), ym, 1)[0]
        list_factor[index] = r
    factor_value[date] = list_factor
factor_value.index = iter_ticker_list
factor = factor_value.T
factor.head()

looping by date, FFT:   0%|          | 0/822 [00:00<?, ?it/s]

0050.TW 2021-01-12 00:00:00 [123.98686928646569, 124.07686532022622, 124.14990168298418, 124.20578074375298, 124.24433583785616, 124.26543167181812, 124.26896465467459, 124.25486315489634, 124.22308768229891, 124.17363099449472, 124.10651812762661, 124.02180635130475, 123.91958504785403, 123.79997551616307, 123.66313070060639, 123.50923484569692, 123.33850307730359, 123.15118091144717, 122.94754369186545, 122.72789595770828, 122.49257074289636, 122.24192880883966, 121.9763578123765, 121.6962714109489, 121.40210830718361, 121.09433123519413, 120.7734258910612, 120.43989981008404, 120.09428119352384, 119.73711768768327, 119.36897511828126, 118.99043618319074, 118.60209910670743, 118.20457625861116, 117.7984927413661, 117.38448494888375, 116.96319910033952, 116.53528975259681, 116.10141829484061, 115.6622514290679, 115.21845964011457, 114.77071565892363, 114.31969292277577, 113.86606403620951, 113.41049923635653, 112.95366486640498, 112.4962218608851, 112.03882424644053, 111.5821176617106

looping by date, FFT:   0%|          | 0/822 [00:08<?, ?it/s]

1733.TW 2021-01-12 00:00:00 [20.864521808046398, 20.89134940115674, 20.91851272037491, 20.946014998558464, 20.973860228855862, 21.002053052461605, 21.03059864195543, 21.059502580678092, 21.088770738610215, 21.118409145233677, 21.148423859866302, 21.178820839970154, 21.20960580794239, 21.24078411690379, 21.272360616005596, 21.30433951577842, 21.336724254048963, 21.369517362950432, 21.40272033755111, 21.436333506622443, 21.470355906063322, 21.50478515549089, 21.53961733850036, 21.57484688708666, 21.61046647070994, 21.646466890474112, 21.682836978873716, 21.719563505548788, 21.7566310894706, 21.794022117962566, 21.831716672941425, 21.869692464742496, 21.907924773871027, 21.946386400998424, 21.98504762549763, 22.02387617278695, 22.062837190725173, 22.101893235273824, 22.141004265614484, 22.180127648880596, 22.219218174633845, 22.258228079185578, 22.297107079833463, 22.335802419052975, 22.374258918652476, 22.41241904386959, 22.450222977355423, 22.48760870296189, 22.524512099216434, 22.56086

LinAlgError: SVD did not converge in Linear Least Squares

In [33]:
start = adj_close.index[0]
end   = adj_close.index[252]
print(start, end)

for ticker in list(adj_close.columns):
    ym = main(adj_close[:252][ticker])

# main(adj_close['0050.TW'][-252:])

2020-01-02 00:00:00 2021-01-13 00:00:00
97.86873308058554
30.269126753076158
68.2074339872203
41.08396186064795
15.400243970300966
26.19354881542524
64.42519920867151
19.570847769253334
31.358779237727937
33.86448468135382
13.874681977772816
18.32335742904397
6.893232531932449
14.47769119627504
16.753123674127625
19.958036958674803
28.4862602864556
36.443084924315215
4.712167518999322
49.134981959367664
61.431469550051744
7.635736169279378
20.951285438870087
15.287632439745696
9.44431126032381
29.277765513434257
59.25641372644367
24.328536641214573
28.1612529173217
116.08609967871239
34.57524920591682
26.910410410702614
14.246623639157741
14.757910780580078
35.84802844686247
223.84521833357496
58.68493071914716
185.7536766193137
106.9787875853091
69.91256627115443
52.40967932012102
14.094879198404028
15.970925287554563
19.769620249617784
12.221528280483572
15.434692659754834
15.028640106044614
14.861562379689387
12.587096644920631
9.062491897545883
54.021370744958965
11.747673472670954

KeyboardInterrupt: 

In [6]:
def getNavSeries(df_now_for_nav, weighting=[0.2] * 5, nav_current = 1):
    df2 = (1 + df_now_for_nav.pct_change()).cumprod()
    df3 = df2.fillna(1)
    df4 = df3*nav_current
    nav_series = np.sum(df4 * weighting, axis=1)
    return nav_series
    
